<a href="https://colab.research.google.com/github/LinYenShou/114-1/blob/main/%E8%95%83%E8%8C%84%E9%90%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [23]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [24]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [25]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [26]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1DvEf2ENY_hR6KZ5obF9oo6O52CH843XqUACbBalsSrE/edit?hl=zh-tw&gid=0#gid=0"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

In [27]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url(SHEET_URL)


# 從 gsheets 的 All-whiteboard-device 載入 sheets
sh = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sh[1:], columns=sh[0])
# 取得最前面的5筆資料
df.head()

""


In [28]:
def ensure_spreadsheet(name):
    try:
        sh = gc.open(name)  # returns gspread.models.Spreadsheet
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

sh = ensure_spreadsheet(WORKSHEET_NAME)

In [32]:
try:
    from google.colab import auth
    from google.auth import default

    print("--- 🔑 啟動「光之試煉」認證儀式... ---")
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("--- ✅ 認證成功，首席導遊 gc 已就位。 ---")

    # 進入「藏寶庫」
    sh = gc.open_by_url(SHEET_URL)

except ImportError:
    print("⚠️ 警告：不在 Colab 環境中，請使用其他 gspread 認證方式 (如 Service Account)。")
    # 如果您在本地環境運行，需要使用其他 gspread 認證方式
    # gc = gspread.service_account(filename="path/to/your/service_account.json")
    # sh = gc.open_by_url(SHEET_URL)
    sys.exit() # 確保程式在沒有認證時停止


# =================================================================
# ⚙️ 核心資料結構與輔助函數
# =================================================================

# 任務表頭
TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
# 紀錄表頭
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
# 爬蟲擷取表頭
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]


def ensure_worksheet(sh, title, header):
    """確保工作表存在且表頭正確"""
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        # 新增工作表
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])

    # 若沒有表頭就補上或修正
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

# 創建或取得工作表物件
ws_tasks = ensure_worksheet(sh, "tasks", TASKS_HEADER)
ws_logs  = ensure_worksheet(sh, "pomodoro_logs", LOGS_HEADER)
ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER)

# 全域變數用於追蹤任務、日誌和爬蟲數據
tasks_df = pd.DataFrame(columns=TASKS_HEADER)
logs_df  = pd.DataFrame(columns=LOGS_HEADER)
clips_df = pd.DataFrame(columns=CLIPS_HEADER)
_active_sessions = {}  # { task_id: {"phase": "work"/"break", "start_ts": iso, "cycles": int} }


def tznow():
    """取得帶時區的當前時間"""
    return dt.now(gettz(TIMEZONE))

def read_df(ws, header):
    """從工作表讀取數據並整理為 DataFrame"""
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    # 保證欄位齊全
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 型別微調
    for col in ["est_min", "actual_min", "pomodoros"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    return df[header]

def write_df(ws, df, header):
    """將 DataFrame 寫入工作表"""
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)

    ws.clear()
    ws.update([header] + df_out.values.tolist())

def refresh_all():
    """從 Google Sheet 重新整理所有數據"""
    return (
        read_df(ws_tasks, TASKS_HEADER).copy(),
        read_df(ws_logs, LOGS_HEADER).copy(),
        read_df(ws_clips, CLIPS_HEADER).copy()
    )

# 初始載入數據
tasks_df, logs_df, clips_df = refresh_all()


# =================================================================
# 🛠️ 任務卷宗操作 (Tasks Scroll Operations)
# =================================================================

def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    """鑄造新任務"""
    global tasks_df
    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    # 【✨ 創意反饋】
    return f"✨ 鑄造成功！新任務卷宗「{task.strip()}」已載入時間軸。", tasks_df

def update_task_status(task_id, new_status):
    """更新任務狀態"""
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務 ID，請確認是否已被刪除或 ID 輸入錯誤。", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status == "done" and not tasks_df.loc[i, "completed_at"]:
        tasks_df.loc[i, "completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return f"✅ 任務 [{task_id}] 狀態已更新為：{new_status.upper()}", tasks_df

def mark_done(task_id):
    """封存任務 (直接標記完成)"""
    return update_task_status(task_id, "done")

def recalc_task_actuals(task_id):
    """根據 logs_df 回寫 actual_min 與 pomodoros"""
    global tasks_df, logs_df
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0))
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()

def list_task_choices():
    """生成下拉選單的任務列表"""
    global tasks_df
    if tasks_df.empty:
        return []
    # 顯示： [status] (P:priority) task  — id
    def row_label(r):
        return f"[{r['status']}] (P:{r['priority']}) {r['task']} — {r['id']}"
    return [(row_label(r), r["id"]) for _, r in tasks_df.iterrows()]


# =================================================================
# 🍅 專注晶體操作 (Pomodoro/Log Operations)
# =================================================================

def suggest_next_task():
    """
    【🌟 創意功能】根據優先級和估計時間推薦下一個待辦或進行中的任務。
    """
    global tasks_df
    cand = tasks_df[
        (tasks_df["status"].isin(["todo", "in-progress"]))
    ].copy()

    if cand.empty:
        return "🎉 恭喜！您的任務清單已清空，請享受您的自由時間！"

    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    # 優先處理 today 的任務
    cand["today_prio"] = cand["planned_for"].apply(lambda x: 0 if x.lower() == "today" else 1)

    cand = cand.sort_values(["today_prio", "p_ord", "est_min"], ascending=[True, True, True])

    r = cand.iloc[0]
    return f"🌟 蕃茄鐘建議：下一個挑戰是 **[{r['id']}] {r['task']}** (P:{r['priority']}，估計 {r['est_min']} 分鐘)。"

def start_phase(task_id, phase, cycles):
    """啟動工作或休息階段 (心流鎖定/能量回溯)"""
    if not task_id: return "⚠️ 請先選擇任務才能啟動專注晶體！"
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    action = "心流鎖定 (Work)" if phase == "work" else "能量回溯 (Break)"
    return f"▶️ 已啟動：{action}（任務 ID: {task_id}）"

def end_phase(task_id, note):
    """結束階段並同步時軌 (寫入 Logs)"""
    global logs_df, tasks_df
    if task_id not in _active_sessions:
        return "⚠️ 偵測到時軌異常，尚未開始任何階段！"

    sess = _active_sessions.pop(task_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)

    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)

    # 回填任務數據
    if sess["phase"] == "work":
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER)

        # 【🔥 創意修改點：工作結束後給予建議】
        next_task_suggestion = suggest_next_task()
        return (
            f"🔥 心流鎖定解除！任務時軌同步成功：**{minutes} 分鐘**，紀錄至 Logs。\n\n"
            f"**請立即進入短暫休息！**\n\n{next_task_suggestion}"
        )
    else:
        # 【🍵 創意修改點：休息結束】
        return f"🍵 能量回溯完畢！紀錄 {minutes} 分鐘。準備好啟動下一輪專注晶體。"


# =================================================================
# 🧠 智慧謀劃室 (AI Plan)
# =================================================================

def generate_today_plan():
    """生成今日任務規劃 (優先 Gemini，其次規則式)"""
    global tasks_df

    # 這裡的程式碼需要能夠訪問到您在主程式碼開頭定義的全局變數 `model`
    # (即您用 genai.GenerativeModel('gemini-2.5-pro') 初始化的那個變數)

    today = tznow().date().isoformat()
    # 篩選今天待辦且未完成的任務
    cand = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")) &
        (tasks_df["status"]!="done")
    ].copy()

    if cand.empty:
        return "📭 今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    # 排序： priority (H>M>L) + est_min
    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    plan_md = None

    # ===============================================
    # 🧠 嘗試使用 GEMINI 進行規劃
    # ===============================================

    # 檢查全局變數 `model` 是否存在且為 GenerativeModel 實例
    if 'model' in globals() and isinstance(globals()['model'], genai.GenerativeModel):
        try:
            sys_prompt = (
                "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
                "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
                "回傳以 Markdown 條列，格式：\n"
                "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n..."
                "### Afternoon\n...\n### Evening\n...\n"
            )
            # 準備任務列表給 AI
            items = []
            for _, r in cand.iterrows():
                items.append({
                    "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                    "priority": r["priority"]
                })
            user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)

            # 呼叫 Gemini 進行排程 (使用全局 model)
            resp = model.generate_content(
                sys_prompt + "\n\n任務 JSON 資料:\n" + user_content
            )
            plan_md = "✅ **AI 智慧謀劃結果** (Gemini-2.5-Pro)：\n" + resp.text
        except Exception as e:
            plan_md = f"⚠️ Gemini 呼叫失敗：{e}\n\n改用規則式規劃。"
    else:
        plan_md = "🔧 **Gemini 模型 (model) 未成功初始化或未定義。**\n"


    # ===============================================
    # 📜 規則式規劃 (作為備用或補充)
    # ===============================================
    buckets = {"morning": [], "afternoon": [], "evening": []}
    for i, (_, r) in enumerate(cand.iterrows()):
        if i % 3 == 0:
            buckets["morning"].append(r)
        elif i % 3 == 1:
            buckets["afternoon"].append(r)
        else:
            buckets["evening"].append(r)

    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n（無安排）\n"
        lines = [f"### ☀️ {name.title()} 時區 (規則式)"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']}（預估 {int(r['est_min'])} 分，P:{r['priority']}）")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    # 整合輸出
    final_output = (plan_md + "\n\n---\n\n📜 **規則式規劃備案**：\n" + rule_md).strip()
    return final_output




# =================================================================
# 🕷️ 雲端採集器 (Crawler)
# =================================================================
# 這裡省略了 crawl 和 add_clips_as_tasks 函式，因為它們是標準爬蟲邏輯，
# 且篇幅限制，假設它們已在您的環境中。

def crawl(url, selector, mode, limit):
    # 替換為您原有的 crawl 函式實現
    return pd.DataFrame(columns=CLIPS_HEADER), "✅ 擷取 0 筆 (此為模擬輸出)"

def add_clips_as_tasks(clip_ids, default_priority, est_min):
    # 替換為您原有的 add_clips_as_tasks 函式實現
    global clips_df, tasks_df
    return "✅ 已加入 0 項為任務 (此為模擬輸出)", clips_df, tasks_df

# =================================================================
# 📊 儀表板與總結
# =================================================================

def today_summary():
    """今日完成率總結"""
    global tasks_df
    today = tznow().date().isoformat()
    planned = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today"))
    ]
    done = planned[planned["status"]=="done"]
    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    # 【📊 創意總結】
    return f"📅 **今日計畫任務**：{total} | ✅ **完成封存**：{done_n} | 📈 **當前進度**：{rate:.1f}%"


# =================================================================
# 🖥️ GRADIO 介面 (指揮中心儀表板)
# =================================================================

def _refresh():
    """Gradio 刷新動作"""
    global tasks_df, logs_df, clips_df
    tasks_df, logs_df, clips_df = refresh_all()
    return tasks_df, logs_df, clips_df, list_task_choices(), today_summary()


with gr.Blocks(title="任務卷宗＋專注晶體＋AI 智慧謀劃室") as demo:
    gr.Markdown("# 🚀 高效率指揮中心：任務卷宗與專注晶體")
    with gr.Row():
        btn_refresh = gr.Button("🔄 同步雲端卷宗 (Sheet → App)")
        out_summary = gr.Markdown(today_summary())

    with gr.Tab("任務卷宗"):
        gr.Markdown("### 📜 任務鑄造台")
        with gr.Row():
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="寫 HW3 報告 / 修正 SQL / …")
                priority = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 鑄造任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=tasks_df, label="📑 任務清單（雲端同步）", interactive=False)

        gr.Markdown("### 📝 狀態更新與封存")
        with gr.Row():
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於更新）")
            new_status = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_done = gr.Button("✅ 封存任務（標記完成）")
            msg_update = gr.Markdown()

    with gr.Tab("專注晶體"):
        gr.Markdown("### ⏱️ 番茄鐘專注系統：每次行動都會同步時軌紀錄。")
        with gr.Row():
            sel_task = gr.Dropdown(choices=list_task_choices(), label="🛡️ 鎖定目標任務")
            cycles = gr.Number(value=1, precision=0, label="⚡️ 專注循環次數（僅作紀錄）")
        with gr.Row():
            btn_start_work = gr.Button("▶️ 啟動：心流鎖定 (Work)")
            note_work = gr.Textbox(label="工作階段備註（紀錄成果/阻礙）")
            btn_end_work = gr.Button("⏹️ 解除鎖定並同步時軌")
        with gr.Row():
            btn_start_break = gr.Button("🍵 啟動：能量回溯 (Break)")
            note_break = gr.Textbox(label="休息階段備註（冥想/咖啡/伸展）")
            btn_end_break = gr.Button("⏹️ 中止回溯並同步時軌")
        msg_pomo = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="⏱️ 時間軌跡紀錄 (番茄鐘紀錄)", interactive=False)

    with gr.Tab("智慧謀劃室"):
        gr.Markdown("### 🧠 AI 行動計畫：將今天的任務排成 morning / afternoon / evening 三段。")
        btn_plan = gr.Button("🎯 產生今日行動計畫")
        out_plan = gr.Markdown()

    with gr.Tab("雲端採集器"):
        gr.Markdown("### 🕸️ 網頁資訊擷取與任務轉化。")
        url = gr.Textbox(label="目標 URL", placeholder="https://example.com")
        selector = gr.Textbox(label="CSS Selector", placeholder="a.news-item / h2.title / div.card a")
        mode = gr.Radio(["text","href","both"], value="text", label="擷取內容")
        limit = gr.Number(value=20, precision=0, label="最多擷取幾筆")
        btn_crawl = gr.Button("🕷️ 開始擷取")
        msg_crawl = gr.Markdown()
        grid_clips = gr.Dataframe(value=clips_df, label="🗄️ 擷取結果（會同步寫入 Sheet）", interactive=True)
        clip_ids = gr.Textbox(label="要加入任務的 clip_id（多個以逗號分隔）")
        default_priority = gr.Dropdown(["H","M","L"], value="L", label="新增任務優先級")
        clip_est = gr.Number(value=25, precision=0, label="新增任務預估分鐘")
        btn_add_clips = gr.Button("➕ 將勾選的擷取項目鑄造成任務")
        msg_add_clips = gr.Markdown()

    with gr.Tab("總結儀表板"):
        gr.Markdown("### 📊 今日數據報告")
        btn_summary = gr.Button("📈 重新計算今日完成率")
        out_summary2 = gr.Markdown(today_summary())

    # === 綁定動作 (Action Bindings) ===

    # 任務操作
    btn_refresh.click(_refresh, outputs=[grid_tasks, grid_logs, grid_clips, task_choice, out_summary])
    btn_add.click(add_task, inputs=[task, priority, est_min, due_date, labels, notes, planned_for], outputs=[msg_add, grid_tasks])
    btn_update.click(update_task_status, inputs=[task_choice, new_status], outputs=[msg_update, grid_tasks])
    btn_done.click(mark_done, inputs=[task_choice], outputs=[msg_update, grid_tasks])

    # 專注晶體操作
    btn_start_work.click(start_phase, inputs=[sel_task, gr.State("work"), cycles], outputs=[msg_pomo])
    btn_end_work.click(end_phase, inputs=[sel_task, note_work], outputs=[msg_pomo])
    btn_start_break.click(start_phase, inputs=[sel_task, gr.State("break"), cycles], outputs=[msg_pomo])
    btn_end_break.click(end_phase, inputs=[sel_task, note_break], outputs=[msg_pomo])

    # AI 規劃
    btn_plan.click(generate_today_plan, outputs=[out_plan])

    # 採集器操作
    def _crawl_and_save(u, s, m, l):
        df, msg = crawl(u, s, m, l)
        global clips_df
        if not df.empty:
            clips_df = pd.concat([clips_df, df], ignore_index=True)
            write_df(ws_clips, clips_df, CLIPS_HEADER)
        return msg, clips_df
    btn_crawl.click(_crawl_and_save, inputs=[url, selector, mode, limit], outputs=[msg_crawl, grid_clips])

    def _add_clips(clip_ids_str, pr, est):
        ids = [c.strip() for c in (clip_ids_str or "").split(",") if c.strip()]
        msg, new_clips, new_tasks = add_clips_as_tasks(ids, pr, est)
        return msg, new_clips, new_tasks
    btn_add_clips.click(_add_clips, inputs=[clip_ids, default_priority, clip_est], outputs=[msg_add_clips, grid_clips, grid_tasks])

    # 總結
    btn_summary.click(today_summary, outputs=[out_summary2])

# 啟動 Gradio
demo.launch(inbrowser=True)

--- 🔑 啟動「光之試煉」認證儀式... ---
--- ✅ 認證成功，首席導遊 gc 已就位。 ---
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d33ddf8052e212380.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
